In [50]:
#BUENO Captura!!!!!!
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pandas as pd
import os
import base64

url = 'https://cndes-web.ses.mir.es/publico/Desaparecidos/'

# Configurar el driver de Selenium (asegúrate de tener ChromeDriver instalado)
driver = webdriver.Chrome()

# Agregar una pausa de 2 segundos después de cargar la página
time.sleep(2)

# Realizar la solicitud HTTP con Selenium
driver.get(url)

#Crear lista
desaparecidos=[]
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.btn.btn-info'))).click()
time.sleep(1)
for i in range(1,2):
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'/html/body/section[1]/div/div/div[{i}]/div/div[3]/div[2]/a'))).click()

    texto_columnas = driver.find_element(By.CLASS_NAME, 'fichaDet').text
    time.sleep(1)
    # Dividir el texto en líneas y procesar cada línea
    lineas = texto_columnas.split('\n')

    # Filtrar las líneas para excluir elementos no deseados
    lineas_filtradas = [linea for linea in lineas if not linea.startswith(('Compartir', 'INFORMAR'))]

    # Crear un diccionario
    info_dict = {'NOMBRE': lineas_filtradas[0]}

    # Recorrer las líneas desde la segunda posición y agregar al diccionario
    for i in range(1, len(lineas_filtradas)-1, 2):
        etiqueta = lineas_filtradas[i]
        valor = lineas_filtradas[i + 1]
        info_dict[etiqueta] = valor
        
    desaparecidos.append(info_dict)
    driver.back()
# Imprimir el diccionario
print(desaparecidos)
#HAY QUE CREAR CON ELEMENTO <div class="localizado">localizado</div> PARA ELIMINAR O CREAR OTRO CAMPO SI APARECE CON LOCALIZADO{ESTADO:LOCALIZADO}


[{'NOMBRE': 'YOSMAIRIS ALCANTARA MORILLO', 'DESAPARECE': '23 de enero de 2024', 'FECHA NACIMIENTO': '19 de agosto de 2008', 'EDAD AL DESAPARECER': '15 años', 'EDAD ACTUAL': '15 años', 'LUGAR DE DESAPARICIÓN': 'ZARAGOZA. ZARAGOZA, ARAGÓN'}]


In [52]:
#buena coge imagenes!!!!!


# Definir la lista de prueba
lista_prueba=[{'NOMBRE': 'HENRY ALEJANDRO JIMENEZ MARIN', 'DESAPARECE': '01 de enero de 2019', 'FECHA NACIMIENTO': '18 de marzo de 1998', 'EDAD AL DESAPARECER': '20 años', 'EDAD ACTUAL': '25 años', 'LUGAR DE DESAPARICIÓN': 'ORIHUELA. ALICANTE/ALACANT, COMUNIDAD VALENCIANA'},{'NOMBRE': 'YOSMAIRIS ALCANTARA MORILLO', 'DESAPARECE': '23 de enero de 2024', 'FECHA NACIMIENTO': '19 de agosto de 2008', 'EDAD AL DESAPARECER': '15 años', 'EDAD ACTUAL': '15 años', 'LUGAR DE DESAPARICIÓN': 'ZARAGOZA. ZARAGOZA, ARAGÓN'}]
    
url = 'https://cndes-web.ses.mir.es/publico/Desaparecidos/Buscar.html'

lista_fallos=[]
# Configurar el driver de Selenium (asegúrate de tener ChromeDriver instalado)
driver_2 = webdriver.Chrome()

# Agregar una pausa de 2 segundos después de cargar la página
time.sleep(2)

# Realizar la solicitud HTTP con Selenium
driver_2.get(url)
WebDriverWait(driver_2, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.btn.btn-info'))).click()
#WebDriverWait(driver_2, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'btn.btnFor.titulo.collapsed'))).click()
WebDriverWait(driver_2, 5).until(EC.element_to_be_clickable((By.ID, 'desp1'))).click()


# Esperar hasta que el elemento con el ID 'nombre' sea visible
#elemento_nombre = WebDriverWait(driver_2, 5).until(EC.visibility_of_element_located((By.ID, 'nombre')))
#elemento_nombre.send_keys(nombre)

#elemento_apellido = WebDriverWait(driver_2, 5).until(EC.visibility_of_element_located((By.ID, 'apellido2')))
#elemento_nombre.send_keys(apellido)

# Resto del código...
for i in lista_prueba:
    nombre = i['NOMBRE'].split(' ')[0]
    a = i['NOMBRE'].split(' ')
    apellido=a[len(a)-1]
    # Esperar hasta que el elemento con el ID 'nombre' sea visible
    elemento_nombre = WebDriverWait(driver_2, 5).until(EC.visibility_of_element_located((By.ID, 'nombre')))
    #elemento_nombre.clear()  # Limpiar el campo de nombre
    elemento_nombre.send_keys(nombre)

    # Esperar hasta que el elemento con el ID 'apellido1' sea visible
    elemento_apellido = WebDriverWait(driver_2, 5).until(EC.visibility_of_element_located((By.ID, 'apellido2')))
    #elemento_apellido.clear()  # Limpiar el campo de apellido
    elemento_apellido.send_keys(apellido)

    try:
        WebDriverWait(driver_2, 10).until(EC.element_to_be_clickable((By.ID, 'enviar'))).click()

        # Pausa para asegurar que la página haya cargado completamente
        time.sleep(4)

        # Obtener la URL de la imagen (reemplaza el xpath con el correcto)
        img_element = driver_2.find_element(By.XPATH, "/html/body/section[2]/div/div[2]/div/div/div[1]/a/img")
        img_data_base64 = img_element.get_attribute("src").split(",")[1]  # Extraer la parte de base64

        # Decodificar y guardar la imagen en la carpeta "imagenes"
        carpeta_imagenes = "imagenes"
        ruta_imagen = os.path.join(carpeta_imagenes, f"{i['NOMBRE']}.png")

        # Guardar la imagen en la carpeta "imagenes"
        os.makedirs(carpeta_imagenes, exist_ok=True)
        with open(ruta_imagen, "wb") as img_file:
            img_file.write(base64.b64decode(img_data_base64))

        # Añadir la ruta de la imagen al diccionario actual
        i['imagen'] = ruta_imagen

    except Exception as e:
        WebDriverWait(driver_2, 5).until(EC.element_to_be_clickable((By.ID, 'buttonAceptarError'))).click()
        print(f"Error al procesar {i['NOMBRE']}")
        time.sleep(2)
        
        elemento_apellido.clear()  # Limpiar el campo de apellido
        elemento_nombre.clear()  # Limpiar el campo de nombre
        #driver_2.find_element(By.XPATH, "/html/body/header/div[1]/div/div[3]/div/div[2]/a").click()#boton de buscar
        #WebDriverWait(driver_2, 5).until(EC.element_to_be_clickable((By.ID, 'desp1'))).click#abre inputs nombre apellidos
        #time.sleep(1)
        nombre = i['NOMBRE'].split(' ')[0]
        a = i['NOMBRE'].split(' ')
        apellido=a[len(a)-1]
        #time.sleep(4)
        # Esperar hasta que el elemento con el ID 'nombre' sea visible
        elemento_nombre = WebDriverWait(driver_2, 5).until(EC.visibility_of_element_located((By.ID, 'nombre')))
        #elemento_nombre.clear()  # Limpiar el campo de nombre
        elemento_nombre.send_keys(nombre)

        # Esperar hasta que el elemento con el ID 'apellido1' sea visible
        elemento_apellido = WebDriverWait(driver_2, 5).until(EC.visibility_of_element_located((By.ID, 'apellido1')))
        #elemento_apellido.clear()  # Limpiar el campo de apellido
        elemento_apellido.send_keys(apellido)
        #time.sleep(2)
        WebDriverWait(driver_2, 10).until(EC.element_to_be_clickable((By.ID, 'enviar'))).click()

        # Pausa para asegurar que la página haya cargado completamente
        time.sleep(4)

        # Obtener la URL de la imagen (reemplaza el xpath con el correcto)
                                                      
        img_element = driver_2.find_element(By.XPATH, "/html/body/section[2]/div/div[2]/div/div/div[1]/a/img")
        img_data_base64 = img_element.get_attribute("src").split(",")[1]  # Extraer la parte de base64

        # Decodificar y guardar la imagen en la carpeta "imagenes"
        carpeta_imagenes = "imagenes"
        ruta_imagen = os.path.join(carpeta_imagenes, f"{i['NOMBRE']}.png")

        # Guardar la imagen en la carpeta "imagenes"
        os.makedirs(carpeta_imagenes, exist_ok=True)
        with open(ruta_imagen, "wb") as img_file:
            img_file.write(base64.b64decode(img_data_base64))

        # Añadir la ruta de la imagen al diccionario actual
        i['imagen'] = ruta_imagen

    finally:
        # Limpiar campos de nombre y apellido1 para una nueva búsqueda
        #time.sleep(2)
        driver_2.find_element(By.XPATH, "/html/body/header/div[1]/div/div[3]/div/div[2]/a").click()
        WebDriverWait(driver_2, 5).until(EC.element_to_be_clickable((By.ID, 'desp1'))).click()

        #elemento_nombre.clear()
        #elemento_apellido.clear()

# Cerrar el navegador después de realizar todas las operaciones
driver_2.quit()

# Imprimir la lista después de cerrar el navegador
print(lista_prueba)
print(lista_fallos)

[{'NOMBRE': 'HENRY ALEJANDRO JIMENEZ MARIN', 'DESAPARECE': '01 de enero de 2019', 'FECHA NACIMIENTO': '18 de marzo de 1998', 'EDAD AL DESAPARECER': '20 años', 'EDAD ACTUAL': '25 años', 'LUGAR DE DESAPARICIÓN': 'ORIHUELA. ALICANTE/ALACANT, COMUNIDAD VALENCIANA', 'imagen': 'imagenes\\HENRY ALEJANDRO JIMENEZ MARIN.png'}, {'NOMBRE': 'YOSMAIRIS ALCANTARA MORILLO', 'DESAPARECE': '23 de enero de 2024', 'FECHA NACIMIENTO': '19 de agosto de 2008', 'EDAD AL DESAPARECER': '15 años', 'EDAD ACTUAL': '15 años', 'LUGAR DE DESAPARICIÓN': 'ZARAGOZA. ZARAGOZA, ARAGÓN', 'imagen': 'imagenes\\YOSMAIRIS ALCANTARA MORILLO.png'}]
[]


In [ ]:
# Tu lista de diccionarios
desaparecidos = [{'NOMBRE': 'LILLY ARIANNA PAPPSZILAGYI', 'DESAPARECE': '12 de enero de 2024', 'FECHA NACIMIENTO': '16 de octubre de 2006', 'EDAD AL DESAPARECER': '17 años', 'EDAD ACTUAL': '17 años', 'LUGAR DE DESAPARICIÓN': 'VICAR. ALMERÍA, ANDALUCÍA', 'imagen': 'imagenes\\LILLY ARIANNA PAPPSZILAGYI.png'}, {'NOMBRE': 'MICHELLE-FRANCISCA MOLINA CHINCHAY', 'DESAPARECE': '12 de enero de 2024', 'FECHA NACIMIENTO': '07 de noviembre de 2008', 'EDAD AL DESAPARECER': '15 años', 'EDAD ACTUAL': '15 años', 'LUGAR DE DESAPARICIÓN': 'MURCIA. MURCIA, MURCIA', 'imagen': 'imagenes\\MICHELLE-FRANCISCA MOLINA CHINCHAY.png'}, {'NOMBRE': 'PAULA MARÍA DORESTE HERNÁNDEZ', 'DESAPARECE': '07 de enero de 2024', 'FECHA NACIMIENTO': '20 de noviembre de 2008', 'EDAD AL DESAPARECER': '15 años', 'EDAD ACTUAL': '15 años', 'LUGAR DE DESAPARICIÓN': 'SANTA CRUZ DE TENERIFE. SANTA CRUZ DE TENERIFE, CANARIAS', 'imagen': 'imagenes\\PAULA MARÍA DORESTE HERNÁNDEZ.png'}, {'NOMBRE': 'JUAN FRANCISCO UGALDE ECHAVE', 'DESAPARECE': '25 de octubre de 2023', 'FECHA NACIMIENTO': '26 de enero de 1967', 'EDAD AL DESAPARECER': '56 años', 'EDAD ACTUAL': '56 años', 'LUGAR DE DESAPARICIÓN': 'ASTIGARRAGA. GIPUZKOA, PAÍS VASCO', 'imagen': 'imagenes\\JUAN FRANCISCO UGALDE ECHAVE.png'}, {'NOMBRE': 'CAMILA REBECCA QUEZADA SIDORKEVICH', 'DESAPARECE': '29 de agosto de 2021', 'FECHA NACIMIENTO': '08 de enero de 2015', 'EDAD AL DESAPARECER': '6 años', 'EDAD ACTUAL': '9 años', 'LUGAR DE DESAPARICIÓN': 'CALDES DE MONTBUI. BARCELONA, CATALUÑA', 'imagen': 'imagenes\\CAMILA REBECCA QUEZADA SIDORKEVICH.png'}, {'NOMBRE': 'CAROLINA GARCIA MAGAN', 'DESAPARECE': '03 de noviembre de 2023', 'FECHA NACIMIENTO': '13 de marzo de 2007', 'EDAD AL DESAPARECER': '16 años', 'EDAD ACTUAL': '16 años', 'LUGAR DE DESAPARICIÓN': 'TRES CANTOS. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\CAROLINA GARCIA MAGAN.png'}, {'NOMBRE': 'PEDRO DAVIA MAESTRE', 'DESAPARECE': '01 de septiembre de 2023', 'FECHA NACIMIENTO': '21 de diciembre de 1989', 'EDAD AL DESAPARECER': '33 años', 'EDAD ACTUAL': '34 años', 'LUGAR DE DESAPARICIÓN': 'ELCHE/ELX. ALICANTE/ALACANT, COMUNIDAD VALENCIANA', 'imagen': 'imagenes\\PEDRO DAVIA MAESTRE.png'}, {'NOMBRE': 'MAXIMILIANO FABIAN LUDVIK', 'DESAPARECE': '27 de agosto de 2023', 'FECHA NACIMIENTO': '12 de marzo de 1994', 'EDAD AL DESAPARECER': '29 años', 'EDAD ACTUAL': '29 años', 'LUGAR DE DESAPARICIÓN': 'MALAGA. MÁLAGA, ANDALUCÍA', 'imagen': 'imagenes\\MAXIMILIANO FABIAN LUDVIK.png'}, {'NOMBRE': 'OLEKSANDER YAKUBENKO', 'DESAPARECE': '28 de julio de 2023', 'FECHA NACIMIENTO': '13 de diciembre de 1969', 'EDAD AL DESAPARECER': '53 años', 'EDAD ACTUAL': '54 años', 'LUGAR DE DESAPARICIÓN': 'COLMENAR VIEJO. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\OLEKSANDER YAKUBENKO.png'}, {'NOMBRE': 'MARIA DEL PILAR OCAMPO DEL RIEGO', 'DESAPARECE': '28 de noviembre de 2023', 'FECHA NACIMIENTO': '22 de julio de 2007', 'EDAD AL DESAPARECER': '16 años', 'EDAD ACTUAL': '16 años', 'LUGAR DE DESAPARICIÓN': 'FISTERRA. A CORUÑA, GALICIA', 'imagen': 'imagenes\\MARIA DEL PILAR OCAMPO DEL RIEGO.png'}, {'NOMBRE': 'ANTONIO FRIAS SORIA', 'DESAPARECE': '23 de abril de 2023', 'FECHA NACIMIENTO': '01 de septiembre de 1945', 'EDAD AL DESAPARECER': '77 años', 'EDAD ACTUAL': '78 años', 'LUGAR DE DESAPARICIÓN': 'MONTCADA I REIXAC. BARCELONA, CATALUÑA', 'imagen': 'imagenes\\ANTONIO FRIAS SORIA.png'}, {'NOMBRE': 'ANA TORRECILLA MARQUEZ', 'DESAPARECE': '01 de octubre de 2023', 'FECHA NACIMIENTO': '25 de enero de 2008', 'EDAD AL DESAPARECER': '15 años', 'EDAD ACTUAL': '15 años', 'LUGAR DE DESAPARICIÓN': 'PARACUELLOS DE JARAMA. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\ANA TORRECILLA MARQUEZ.png'}, {'NOMBRE': 'MUSTAPHA EL KARCHAOUI', 'DESAPARECE': '17 de agosto de 2018', 'FECHA NACIMIENTO': '02 de abril de 1991', 'EDAD AL DESAPARECER': '27 años', 'EDAD ACTUAL': '32 años', 'LUGAR DE DESAPARICIÓN': 'POBLA (SA). ILLES BALEARS, BALEARES', 'imagen': 'imagenes\\MUSTAPHA EL KARCHAOUI.png'}, {'NOMBRE': 'FATIMA ZHORA EL HABIB', 'DESAPARECE': '03 de diciembre de 2023', 'FECHA NACIMIENTO': '27 de agosto de 2006', 'EDAD AL DESAPARECER': '17 años', 'EDAD ACTUAL': '17 años', 'LUGAR DE DESAPARICIÓN': 'MADRID. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\FATIMA ZHORA EL HABIB.png'}, {'NOMBRE': 'ANIANO ORIVE MARTINEZ', 'DESAPARECE': '19 de octubre de 2022', 'FECHA NACIMIENTO': '09 de febrero de 1940', 'EDAD AL DESAPARECER': '82 años', 'EDAD ACTUAL': '83 años', 'LUGAR DE DESAPARICIÓN': 'VALLE DE TRAPAGA-TRAPAGARAN. BIZKAIA, PAÍS VASCO', 'imagen': 'imagenes\\ANIANO ORIVE MARTINEZ.png'}, {'NOMBRE': 'SANTIAGO GALLEGO ESCOBAR', 'DESAPARECE': '28 de julio de 2023', 'FECHA NACIMIENTO': '22 de marzo de 1977', 'EDAD AL DESAPARECER': '46 años', 'EDAD ACTUAL': '46 años', 'LUGAR DE DESAPARICIÓN': 'PALMA DE MALLORCA. ILLES BALEARS, BALEARES', 'imagen': 'imagenes\\SANTIAGO GALLEGO ESCOBAR.png'}, {'NOMBRE': 'MARIA TRINIDAD GUTIERREZ LENDINEZ', 'DESAPARECE': '11 de agosto de 2023', 'FECHA NACIMIENTO': '07 de noviembre de 1982', 'EDAD AL DESAPARECER': '40 años', 'EDAD ACTUAL': '41 años', 'LUGAR DE DESAPARICIÓN': 'ARCOS DE LA FRONTERA. CÁDIZ, ANDALUCÍA', 'imagen': 'imagenes\\MARIA TRINIDAD GUTIERREZ LENDINEZ.png'}, {'NOMBRE': 'NOELIA RAFET NUÑEZ', 'DESAPARECE': '09 de diciembre de 2023', 'FECHA NACIMIENTO': '11 de mayo de 1985', 'EDAD AL DESAPARECER': '38 años', 'EDAD ACTUAL': '38 años', 'LUGAR DE DESAPARICIÓN': 'RIBA-ROJA DE TURIA. VALENCIA/VALÈNCIA, COMUNIDAD VALENCIANA', 'imagen': 'imagenes\\NOELIA RAFET NUÑEZ.png'}, {'NOMBRE': 'OLEKSANDER YAKUBENKO', 'DESAPARECE': '28 de julio de 2023', 'FECHA NACIMIENTO': '13 de diciembre de 1969', 'EDAD AL DESAPARECER': '53 años', 'EDAD ACTUAL': '54 años', 'LUGAR DE DESAPARICIÓN': 'COLMENAR VIEJO. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\OLEKSANDER YAKUBENKO.png'}, {'NOMBRE': 'SARAY RUBIO MORALES', 'DESAPARECE': '01 de octubre de 2023', 'FECHA NACIMIENTO': '26 de agosto de 2008', 'EDAD AL DESAPARECER': '15 años', 'EDAD ACTUAL': '15 años', 'LUGAR DE DESAPARICIÓN': 'PARACUELLOS DE JARAMA. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\SARAY RUBIO MORALES.png'}, {'NOMBRE': 'ABOUBACAR SIDIBE', 'DESAPARECE': '04 de junio de 2023', 'FECHA NACIMIENTO': '21 de junio de 2009', 'EDAD AL DESAPARECER': '13 años', 'EDAD ACTUAL': '14 años', 'LUGAR DE DESAPARICIÓN': 'TACORONTE. SANTA CRUZ DE TENERIFE, CANARIAS', 'imagen': 'imagenes\\ABOUBACAR SIDIBE.png'}, {'NOMBRE': 'BRITANY ODALIS GARCIA CARRERA', 'DESAPARECE': '01 de diciembre de 2023', 'FECHA NACIMIENTO': '22 de mayo de 2008', 'EDAD AL DESAPARECER': '15 años', 'EDAD ACTUAL': '15 años', 'LUGAR DE DESAPARICIÓN': 'PARACUELLOS DE JARAMA. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\BRITANY ODALIS GARCIA CARRERA.png'}, {'NOMBRE': 'JESUS GIJON GARCIA', 'DESAPARECE': '24 de septiembre de 2023', 'FECHA NACIMIENTO': '10 de marzo de 2006', 'EDAD AL DESAPARECER': '17 años', 'EDAD ACTUAL': '17 años', 'LUGAR DE DESAPARICIÓN': 'POBLA DE VALLBONA (LA). VALENCIA/VALÈNCIA, COMUNIDAD VALENCIANA', 'imagen': 'imagenes\\JESUS GIJON GARCIA.png'}, {'NOMBRE': 'NAIALA BENÍTEZ QUIROS', 'DESAPARECE': '09 de septiembre de 2023', 'FECHA NACIMIENTO': '16 de agosto de 2007', 'EDAD AL DESAPARECER': '16 años', 'EDAD ACTUAL': '16 años', 'LUGAR DE DESAPARICIÓN': 'ADEJE. SANTA CRUZ DE TENERIFE, CANARIAS', 'imagen': 'imagenes\\NAIALA BENÍTEZ QUIROS.png'}, {'NOMBRE': 'ALINA STEFANIA PORTAL CHAUPIZ', 'DESAPARECE': '19 de noviembre de 2023', 'FECHA NACIMIENTO': '05 de mayo de 2007', 'EDAD AL DESAPARECER': '16 años', 'EDAD ACTUAL': '16 años', 'LUGAR DE DESAPARICIÓN': 'ALCOBENDAS. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\ALINA STEFANIA PORTAL CHAUPIZ.png'}, {'NOMBRE': 'YACINE BENZAAMA', 'DESAPARECE': '16 de octubre de 2023', 'FECHA NACIMIENTO': '23 de marzo de 2008', 'EDAD AL DESAPARECER': '15 años', 'EDAD ACTUAL': '15 años', 'LUGAR DE DESAPARICIÓN': 'REAL DE GANDIA. VALENCIA/VALÈNCIA, COMUNIDAD VALENCIANA', 'imagen': 'imagenes\\YACINE BENZAAMA.png'}, {'NOMBRE': 'PEDRO JUAN BLANCO HERNANDEZ', 'DESAPARECE': '29 de agosto de 2023', 'FECHA NACIMIENTO': '04 de mayo de 1971', 'EDAD AL DESAPARECER': '52 años', 'EDAD ACTUAL': '52 años', 'LUGAR DE DESAPARICIÓN': 'SANTA CRUZ DE LA PALMA. SANTA CRUZ DE TENERIFE, CANARIAS', 'imagen': 'imagenes\\PEDRO JUAN BLANCO HERNANDEZ.png'}, {'NOMBRE': 'LUCIO CENTURION ACUÑA', 'DESAPARECE': '06 de diciembre de 2023', 'FECHA NACIMIENTO': '19 de abril de 1995', 'EDAD AL DESAPARECER': '28 años', 'EDAD ACTUAL': '28 años', 'LUGAR DE DESAPARICIÓN': 'MANZANARES EL REAL. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\LUCIO CENTURION ACUÑA.png'}, {'NOMBRE': 'DAMIAN LASTRA FERNÁNDEZ', 'DESAPARECE': '06 de abril de 2022', 'FECHA NACIMIENTO': '01 de octubre de 1951', 'EDAD AL DESAPARECER': '70 años', 'EDAD ACTUAL': '72 años', 'LUGAR DE DESAPARICIÓN': 'AVILES. ASTURIAS, ASTURIAS', 'imagen': 'imagenes\\DAMIAN LASTRA FERNÁNDEZ.png'}, {'NOMBRE': 'IDELFONSO MEDRANO FERNANDEZ', 'DESAPARECE': '06 de octubre de 2021', 'FECHA NACIMIENTO': '28 de febrero de 1967', 'EDAD AL DESAPARECER': '54 años', 'EDAD ACTUAL': '56 años', 'LUGAR DE DESAPARICIÓN': 'MUTXAMEL. ALICANTE/ALACANT, COMUNIDAD VALENCIANA', 'imagen': 'imagenes\\IDELFONSO MEDRANO FERNANDEZ.png'}, {'NOMBRE': 'RACHID BIYAD', 'DESAPARECE': '17 de julio de 2019', 'FECHA NACIMIENTO': '25 de marzo de 2003', 'EDAD AL DESAPARECER': '16 años', 'EDAD ACTUAL': '20 años', 'LUGAR DE DESAPARICIÓN': 'CORTECONCEPCION. HUELVA, ANDALUCÍA', 'imagen': 'imagenes\\RACHID BIYAD.png'}, {'NOMBRE': 'JOSE MANUEL GUTIERREZ CASO', 'DESAPARECE': '26 de noviembre de 2023', 'FECHA NACIMIENTO': '30 de julio de 1947', 'EDAD AL DESAPARECER': '76 años', 'EDAD ACTUAL': '76 años', 'LUGAR DE DESAPARICIÓN': 'LINEA DE LA CONCEPCION (LA). CÁDIZ, ANDALUCÍA', 'imagen': 'imagenes\\JOSE MANUEL GUTIERREZ CASO.png'}, {'NOMBRE': 'FRANCISCO CORTES CANO', 'DESAPARECE': '27 de junio de 2023', 'FECHA NACIMIENTO': '23 de febrero de 1958', 'EDAD AL DESAPARECER': '65 años', 'EDAD ACTUAL': '65 años', 'LUGAR DE DESAPARICIÓN': 'MOLLINA. MÁLAGA, ANDALUCÍA', 'imagen': 'imagenes\\FRANCISCO CORTES CANO.png'}, {'NOMBRE': 'DOMINGO MARTINEZ FERNANDEZ', 'DESAPARECE': '03 de octubre de 2023', 'FECHA NACIMIENTO': '08 de julio de 1941', 'EDAD AL DESAPARECER': '82 años', 'EDAD ACTUAL': '82 años', 'LUGAR DE DESAPARICIÓN': 'ARUCAS. LAS PALMAS, CANARIAS', 'imagen': 'imagenes\\DOMINGO MARTINEZ FERNANDEZ.png'}, {'NOMBRE': 'NASSER FALCON', 'DESAPARECE': '02 de julio de 2023', 'FECHA NACIMIENTO': '30 de julio de 2010', 'EDAD AL DESAPARECER': '12 años', 'EDAD ACTUAL': '13 años', 'LUGAR DE DESAPARICIÓN': 'MONTILLA. CÓRDOBA, ANDALUCÍA', 'imagen': 'imagenes\\NASSER FALCON.png'}, {'NOMBRE': 'CHAHIRA HAFIDA CHOURAK', 'DESAPARECE': '08 de diciembre de 2022', 'FECHA NACIMIENTO': '18 de diciembre de 1998', 'EDAD AL DESAPARECER': '23 años', 'EDAD ACTUAL': '25 años', 'LUGAR DE DESAPARICIÓN': 'ARGELIA', 'imagen': 'imagenes\\CHAHIRA HAFIDA CHOURAK.png'}, {'NOMBRE': 'RIDA ALLAM', 'DESAPARECE': '27 de octubre de 2021', 'FECHA NACIMIENTO': '01 de junio de 2006', 'EDAD AL DESAPARECER': '15 años', 'EDAD ACTUAL': '17 años', 'LUGAR DE DESAPARICIÓN': 'SERON. ALMERÍA, ANDALUCÍA', 'imagen': 'imagenes\\RIDA ALLAM.png'}, {'NOMBRE': 'CARLOS ALBERTO SANTOS DE LOS REYES', 'DESAPARECE': '28 de diciembre de 2023', 'FECHA NACIMIENTO': '02 de marzo de 2006', 'EDAD AL DESAPARECER': '17 años', 'EDAD ACTUAL': '17 años', 'LUGAR DE DESAPARICIÓN': 'MADRID. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\CARLOS ALBERTO SANTOS DE LOS REYES.png'}, {'NOMBRE': 'DANIELA ALEXANDRA SUAREZ BARBA', 'DESAPARECE': '29 de diciembre de 2023', 'FECHA NACIMIENTO': '25 de febrero de 2007', 'EDAD AL DESAPARECER': '16 años', 'EDAD ACTUAL': '16 años', 'LUGAR DE DESAPARICIÓN': 'MADRID. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\DANIELA ALEXANDRA SUAREZ BARBA.png'}, {'NOMBRE': 'LUCIO CENTURION ACUÑA', 'DESAPARECE': '06 de diciembre de 2023', 'FECHA NACIMIENTO': '19 de abril de 1995', 'EDAD AL DESAPARECER': '28 años', 'EDAD ACTUAL': '28 años', 'LUGAR DE DESAPARICIÓN': 'MANZANARES EL REAL. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\LUCIO CENTURION ACUÑA.png'}, {'NOMBRE': 'AKHLOUF LAKHLOUFI', 'DESAPARECE': '12 de julio de 2023', 'FECHA NACIMIENTO': '28 de enero de 1990', 'EDAD AL DESAPARECER': '33 años', 'EDAD ACTUAL': '33 años', 'LUGAR DE DESAPARICIÓN': 'ALMERÍA, ANDALUCÍA', 'imagen': 'imagenes\\AKHLOUF LAKHLOUFI.png'}, {'NOMBRE': 'ANA TORRECILLA MARQUEZ', 'DESAPARECE': '01 de octubre de 2023', 'FECHA NACIMIENTO': '25 de enero de 2008', 'EDAD AL DESAPARECER': '15 años', 'EDAD ACTUAL': '15 años', 'LUGAR DE DESAPARICIÓN': 'PARACUELLOS DE JARAMA. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\ANA TORRECILLA MARQUEZ.png'}, {'NOMBRE': 'JOSE CARLOS ORTEGA GIL', 'DESAPARECE': '01 de febrero de 2021', 'FECHA NACIMIENTO': '10 de septiembre de 1970', 'EDAD AL DESAPARECER': '50 años', 'EDAD ACTUAL': '53 años', 'LUGAR DE DESAPARICIÓN': 'ALCORCON. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\JOSE CARLOS ORTEGA GIL.png'}, {'NOMBRE': 'DIANA MORENO MOTE', 'DESAPARECE': '13 de enero de 2023', 'FECHA NACIMIENTO': '04 de abril de 2006', 'EDAD AL DESAPARECER': '16 años', 'EDAD ACTUAL': '17 años', 'LUGAR DE DESAPARICIÓN': 'PARACUELLOS DE JARAMA. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\DIANA MORENO MOTE.png'}, {'NOMBRE': 'EMMANUEL SORIA', 'DESAPARECE': '27 de agosto de 2023', 'FECHA NACIMIENTO': '06 de octubre de 1988', 'EDAD AL DESAPARECER': '34 años', 'EDAD ACTUAL': '35 años', 'LUGAR DE DESAPARICIÓN': 'MALAGA. MÁLAGA, ANDALUCÍA', 'imagen': 'imagenes\\EMMANUEL SORIA.png'}, {'NOMBRE': 'SARAY RUBIO MORALES', 'DESAPARECE': '01 de octubre de 2023', 'FECHA NACIMIENTO': '26 de agosto de 2008', 'EDAD AL DESAPARECER': '15 años', 'EDAD ACTUAL': '15 años', 'LUGAR DE DESAPARICIÓN': 'PARACUELLOS DE JARAMA. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\SARAY RUBIO MORALES.png'}, {'NOMBRE': 'MIGUEL IBARRA JIMENEZ', 'DESAPARECE': '14 de mayo de 2023', 'FECHA NACIMIENTO': '26 de agosto de 1961', 'EDAD AL DESAPARECER': '61 años', 'EDAD ACTUAL': '62 años', 'LUGAR DE DESAPARICIÓN': 'CAZALILLA. JAÉN, ANDALUCÍA', 'imagen': 'imagenes\\MIGUEL IBARRA JIMENEZ.png'}, {'NOMBRE': 'RAFFAELE DI NARDO', 'DESAPARECE': '02 de marzo de 2022', 'FECHA NACIMIENTO': '04 de agosto de 1983', 'EDAD AL DESAPARECER': '38 años', 'EDAD ACTUAL': '40 años', 'LUGAR DE DESAPARICIÓN': 'ADEJE. SANTA CRUZ DE TENERIFE, CANARIAS', 'imagen': 'imagenes\\RAFFAELE DI NARDO.png'}, {'NOMBRE': 'FRANCISCO JOSÉ AGÜERA CLAVIJO', 'DESAPARECE': '28 de abril de 2020', 'FECHA NACIMIENTO': '24 de mayo de 1960', 'EDAD AL DESAPARECER': '59 años', 'EDAD ACTUAL': '63 años', 'LUGAR DE DESAPARICIÓN': 'COIN. MÁLAGA, ANDALUCÍA', 'imagen': 'imagenes\\FRANCISCO JOSÉ AGÜERA CLAVIJO.png'}, {'NOMBRE': 'MARWA TOUATI', 'DESAPARECE': '08 de diciembre de 2022', 'FECHA NACIMIENTO': '31 de octubre de 2013', 'EDAD AL DESAPARECER': '9 años', 'EDAD ACTUAL': '10 años', 'LUGAR DE DESAPARICIÓN': 'ESPAÑA', 'imagen': 'imagenes\\MARWA TOUATI.png'}, {'NOMBRE': 'VICENTE GOMERA FLORES', 'DESAPARECE': '17 de marzo de 2023', 'FECHA NACIMIENTO': '20 de agosto de 1977', 'EDAD AL DESAPARECER': '45 años', 'EDAD ACTUAL': '46 años', 'LUGAR DE DESAPARICIÓN': 'JAEN. JAÉN, ANDALUCÍA', 'imagen': 'imagenes\\VICENTE GOMERA FLORES.png'}, {'NOMBRE': 'FRANCISCO CANO CANO', 'DESAPARECE': '13 de agosto de 2022', 'FECHA NACIMIENTO': '24 de diciembre de 1944', 'EDAD AL DESAPARECER': '77 años', 'EDAD ACTUAL': '79 años', 'LUGAR DE DESAPARICIÓN': 'BARRIOS (LOS). CÁDIZ, ANDALUCÍA', 'imagen': 'imagenes\\FRANCISCO CANO CANO.png'}, {'NOMBRE': 'ABRAHAN HAG PAULETE', 'DESAPARECE': '10 de julio de 2023', 'FECHA NACIMIENTO': '23 de mayo de 2007', 'EDAD AL DESAPARECER': '16 años', 'EDAD ACTUAL': '16 años', 'LUGAR DE DESAPARICIÓN': 'TORRELODONES. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\ABRAHAN HAG PAULETE.png'}, {'NOMBRE': 'PETRUTA MIOARA DINU', 'DESAPARECE': '22 de octubre de 2023', 'FECHA NACIMIENTO': '18 de noviembre de 2007', 'EDAD AL DESAPARECER': '15 años', 'EDAD ACTUAL': '16 años', 'LUGAR DE DESAPARICIÓN': 'MARCHENA. SEVILLA, ANDALUCÍA', 'imagen': 'imagenes\\PETRUTA MIOARA DINU.png'}, {'NOMBRE': 'JESUS BAYANO ALMAGRO', 'DESAPARECE': '09 de julio de 2023', 'FECHA NACIMIENTO': '29 de septiembre de 2006', 'EDAD AL DESAPARECER': '16 años', 'EDAD ACTUAL': '17 años', 'LUGAR DE DESAPARICIÓN': 'ESCORIAL (EL). MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\JESUS BAYANO ALMAGRO.png'}, {'NOMBRE': 'MARIA DEL PILAR OCAMPO DEL RIEGO', 'DESAPARECE': '28 de noviembre de 2023', 'FECHA NACIMIENTO': '22 de julio de 2007', 'EDAD AL DESAPARECER': '16 años', 'EDAD ACTUAL': '16 años', 'LUGAR DE DESAPARICIÓN': 'FISTERRA. A CORUÑA, GALICIA', 'imagen': 'imagenes\\MARIA DEL PILAR OCAMPO DEL RIEGO.png'}, {'NOMBRE': 'FILIP ROBAK', 'DESAPARECE': '09 de enero de 2024', 'FECHA NACIMIENTO': '10 de febrero de 2006', 'EDAD AL DESAPARECER': '17 años', 'EDAD ACTUAL': '17 años', 'LUGAR DE DESAPARICIÓN': 'MADRID. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\FILIP ROBAK.png'}, {'NOMBRE': 'ANTONIO SANCHEZ CASTILLO', 'DESAPARECE': '17 de septiembre de 2022', 'FECHA NACIMIENTO': '16 de diciembre de 1997', 'EDAD AL DESAPARECER': '24 años', 'EDAD ACTUAL': '26 años', 'LUGAR DE DESAPARICIÓN': 'ZARAGOZA. ZARAGOZA, ARAGÓN', 'imagen': 'imagenes\\ANTONIO SANCHEZ CASTILLO.png'}, {'NOMBRE': 'MOHAMED EMBAREC SIDAHMED ALI', 'DESAPARECE': '11 de septiembre de 2023', 'FECHA NACIMIENTO': '09 de diciembre de 1998', 'EDAD AL DESAPARECER': '24 años', 'EDAD ACTUAL': '25 años', 'LUGAR DE DESAPARICIÓN': 'OVIEDO. ASTURIAS, ASTURIAS', 'imagen': 'imagenes\\MOHAMED EMBAREC SIDAHMED ALI.png'}, {'NOMBRE': 'LORENA DE RAFAEL CINTAS', 'DESAPARECE': '26 de octubre de 2023', 'FECHA NACIMIENTO': '13 de febrero de 1995', 'EDAD AL DESAPARECER': '28 años', 'EDAD ACTUAL': '28 años', 'LUGAR DE DESAPARICIÓN': 'AREVALO. ÁVILA, CASTILLA Y LEÓN', 'imagen': 'imagenes\\LORENA DE RAFAEL CINTAS.png'}, {'NOMBRE': 'OLIVER HEISE', 'DESAPARECE': '04 de agosto de 2022', 'FECHA NACIMIENTO': '12 de mayo de 1999', 'EDAD AL DESAPARECER': '23 años', 'EDAD ACTUAL': '24 años', 'LUGAR DE DESAPARICIÓN': 'SANTA CRUZ DE TENERIFE. SANTA CRUZ DE TENERIFE, CANARIAS', 'imagen': 'imagenes\\OLIVER HEISE.png'}, {'NOMBRE': 'NASSER BOUDASDASS', 'DESAPARECE': '03 de septiembre de 2020', 'FECHA NACIMIENTO': '01 de febrero de 1976', 'EDAD AL DESAPARECER': '44 años', 'EDAD ACTUAL': '47 años', 'LUGAR DE DESAPARICIÓN': 'CALAF. BARCELONA, CATALUÑA', 'imagen': 'imagenes\\NASSER BOUDASDASS.png'}, {'NOMBRE': 'FRANCISCO CORTES CANO', 'DESAPARECE': '27 de junio de 2023', 'FECHA NACIMIENTO': '23 de febrero de 1958', 'EDAD AL DESAPARECER': '65 años', 'EDAD ACTUAL': '65 años', 'LUGAR DE DESAPARICIÓN': 'MOLLINA. MÁLAGA, ANDALUCÍA', 'imagen': 'imagenes\\FRANCISCO CORTES CANO.png'}, {'NOMBRE': 'DAMIAN LASTRA FERNÁNDEZ', 'DESAPARECE': '06 de abril de 2022', 'FECHA NACIMIENTO': '01 de octubre de 1951', 'EDAD AL DESAPARECER': '70 años', 'EDAD ACTUAL': '72 años', 'LUGAR DE DESAPARICIÓN': 'AVILES. ASTURIAS, ASTURIAS', 'imagen': 'imagenes\\DAMIAN LASTRA FERNÁNDEZ.png'}, {'NOMBRE': 'ABRAHAM RIVAS BARRULL', 'DESAPARECE': '14 de septiembre de 2023', 'FECHA NACIMIENTO': '17 de octubre de 2007', 'EDAD AL DESAPARECER': '15 años', 'EDAD ACTUAL': '16 años', 'LUGAR DE DESAPARICIÓN': 'MARCHENA. SEVILLA, ANDALUCÍA', 'imagen': 'imagenes\\ABRAHAM RIVAS BARRULL.png'}, {'NOMBRE': 'ALBA RODRIGUEZ PEREZ', 'DESAPARECE': '04 de octubre de 2023', 'FECHA NACIMIENTO': '08 de mayo de 2007', 'EDAD AL DESAPARECER': '16 años', 'EDAD ACTUAL': '16 años', 'LUGAR DE DESAPARICIÓN': 'TORRELODONES. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\ALBA RODRIGUEZ PEREZ.png'}, {'NOMBRE': 'JUAN FRANCISCO UGALDE ECHAVE', 'DESAPARECE': '25 de octubre de 2023', 'FECHA NACIMIENTO': '26 de enero de 1967', 'EDAD AL DESAPARECER': '56 años', 'EDAD ACTUAL': '56 años', 'LUGAR DE DESAPARICIÓN': 'ASTIGARRAGA. GIPUZKOA, PAÍS VASCO', 'imagen': 'imagenes\\JUAN FRANCISCO UGALDE ECHAVE.png'}, {'NOMBRE': 'LUNA AURORA MORALES GARCIA', 'DESAPARECE': '15 de enero de 2024', 'FECHA NACIMIENTO': '03 de septiembre de 2006', 'EDAD AL DESAPARECER': '17 años', 'EDAD ACTUAL': '17 años', 'LUGAR DE DESAPARICIÓN': 'TORRELODONES. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\LUNA AURORA MORALES GARCIA.png'}, {'NOMBRE': 'JUAN ANTONIO GOMEZ ALARCON', 'DESAPARECE': '20 de julio de 2010', 'FECHA NACIMIENTO': '22 de mayo de 1978', 'EDAD AL DESAPARECER': '32 años', 'EDAD ACTUAL': '45 años', 'LUGAR DE DESAPARICIÓN': 'MIJAS. MÁLAGA, ANDALUCÍA', 'imagen': 'imagenes\\JUAN ANTONIO GOMEZ ALARCON.png'}, {'NOMBRE': 'ROSA MARIA GIMENEZ WANGTIANG', 'DESAPARECE': '01 de enero de 2024', 'FECHA NACIMIENTO': '13 de octubre de 1939', 'EDAD AL DESAPARECER': '84 años', 'EDAD ACTUAL': '84 años', 'LUGAR DE DESAPARICIÓN': 'ZARAGOZA. ZARAGOZA, ARAGÓN', 'imagen': 'imagenes\\ROSA MARIA GIMENEZ WANGTIANG.png'}, {'NOMBRE': 'EMILIO UGARTE ARRUZA', 'DESAPARECE': '10 de febrero de 2014', 'FECHA NACIMIENTO': '16 de septiembre de 1965', 'EDAD AL DESAPARECER': '48 años', 'EDAD ACTUAL': '58 años', 'LUGAR DE DESAPARICIÓN': 'GETXO. BIZKAIA, PAÍS VASCO', 'imagen': 'imagenes\\EMILIO UGARTE ARRUZA.png'}, {'NOMBRE': 'PASCUAL ROBLEDILLO CHISVERT', 'DESAPARECE': '14 de septiembre de 2020', 'FECHA NACIMIENTO': '18 de marzo de 1978', 'EDAD AL DESAPARECER': '42 años', 'EDAD ACTUAL': '45 años', 'LUGAR DE DESAPARICIÓN': 'PICANYA. VALENCIA/VALÈNCIA, COMUNIDAD VALENCIANA', 'imagen': 'imagenes\\PASCUAL ROBLEDILLO CHISVERT.png'}, {'NOMBRE': 'ALFONSO SEVILLA PORTILLO', 'DESAPARECE': '24 de junio de 2022', 'FECHA NACIMIENTO': '01 de febrero de 1952', 'EDAD AL DESAPARECER': '70 años', 'EDAD ACTUAL': '71 años', 'LUGAR DE DESAPARICIÓN': 'ROQUETAS DE MAR. ALMERÍA, ANDALUCÍA', 'imagen': 'imagenes\\ALFONSO SEVILLA PORTILLO.png'}, {'NOMBRE': 'IDELFONSO MEDRANO FERNANDEZ', 'DESAPARECE': '06 de octubre de 2021', 'FECHA NACIMIENTO': '28 de febrero de 1967', 'EDAD AL DESAPARECER': '54 años', 'EDAD ACTUAL': '56 años', 'LUGAR DE DESAPARICIÓN': 'MUTXAMEL. ALICANTE/ALACANT, COMUNIDAD VALENCIANA', 'imagen': 'imagenes\\IDELFONSO MEDRANO FERNANDEZ.png'}, {'NOMBRE': 'ALI MAMAN SALEY', 'DESAPARECE': '28 de agosto de 2021', 'FECHA NACIMIENTO': '24 de marzo de 1981', 'EDAD AL DESAPARECER': '40 años', 'EDAD ACTUAL': '42 años', 'LUGAR DE DESAPARICIÓN': 'MURCIA. MURCIA, MURCIA', 'imagen': 'imagenes\\ALI MAMAN SALEY.png'}, {'NOMBRE': 'RAFFAELE DI NARDO', 'DESAPARECE': '02 de marzo de 2022', 'FECHA NACIMIENTO': '04 de agosto de 1983', 'EDAD AL DESAPARECER': '38 años', 'EDAD ACTUAL': '40 años', 'LUGAR DE DESAPARICIÓN': 'ADEJE. SANTA CRUZ DE TENERIFE, CANARIAS', 'imagen': 'imagenes\\RAFFAELE DI NARDO.png'}, {'NOMBRE': 'FILIP ROBAK', 'DESAPARECE': '09 de enero de 2024', 'FECHA NACIMIENTO': '10 de febrero de 2006', 'EDAD AL DESAPARECER': '17 años', 'EDAD ACTUAL': '17 años', 'LUGAR DE DESAPARICIÓN': 'MADRID. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\FILIP ROBAK.png'}, {'NOMBRE': 'OUMNIA MSALEK', 'DESAPARECE': '22 de febrero de 2022', 'FECHA NACIMIENTO': '07 de diciembre de 2017', 'EDAD AL DESAPARECER': '4 años', 'EDAD ACTUAL': '6 años', 'LUGAR DE DESAPARICIÓN': 'FRANCIA', 'imagen': 'imagenes\\OUMNIA MSALEK.png'}, {'NOMBRE': 'SARAY RUBIO MORALES', 'DESAPARECE': '01 de octubre de 2023', 'FECHA NACIMIENTO': '26 de agosto de 2008', 'EDAD AL DESAPARECER': '15 años', 'EDAD ACTUAL': '15 años', 'LUGAR DE DESAPARICIÓN': 'PARACUELLOS DE JARAMA. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\SARAY RUBIO MORALES.png'}, {'NOMBRE': 'DIANA MORENO MOTE', 'DESAPARECE': '13 de enero de 2023', 'FECHA NACIMIENTO': '04 de abril de 2006', 'EDAD AL DESAPARECER': '16 años', 'EDAD ACTUAL': '17 años', 'LUGAR DE DESAPARICIÓN': 'PARACUELLOS DE JARAMA. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\DIANA MORENO MOTE.png'}, {'NOMBRE': 'EMMANUEL SORIA', 'DESAPARECE': '27 de agosto de 2023', 'FECHA NACIMIENTO': '06 de octubre de 1988', 'EDAD AL DESAPARECER': '34 años', 'EDAD ACTUAL': '35 años', 'LUGAR DE DESAPARICIÓN': 'MALAGA. MÁLAGA, ANDALUCÍA', 'imagen': 'imagenes\\EMMANUEL SORIA.png'}, {'NOMBRE': 'SARA MICHELLE JORDAN MONTAÑO', 'DESAPARECE': '04 de noviembre de 2023', 'FECHA NACIMIENTO': '22 de octubre de 2006', 'EDAD AL DESAPARECER': '17 años', 'EDAD ACTUAL': '17 años', 'LUGAR DE DESAPARICIÓN': 'PARACUELLOS DE JARAMA. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\SARA MICHELLE JORDAN MONTAÑO.png'}, {'NOMBRE': 'SELENE HEREDIA BERMUDEZ', 'DESAPARECE': '15 de enero de 2024', 'FECHA NACIMIENTO': '21 de octubre de 2006', 'EDAD AL DESAPARECER': '17 años', 'EDAD ACTUAL': '17 años', 'LUGAR DE DESAPARICIÓN': 'TOLEDO. TOLEDO, CASTILLA LA MANCHA', 'imagen': 'imagenes\\SELENE HEREDIA BERMUDEZ.png'}, {'NOMBRE': 'MARIA CARMEN CAMARA LOPEZ', 'DESAPARECE': '13 de enero de 2024', 'FECHA NACIMIENTO': '25 de septiembre de 1958', 'EDAD AL DESAPARECER': '65 años', 'EDAD ACTUAL': '65 años', 'LUGAR DE DESAPARICIÓN': 'POBLA DE VALLBONA (LA). VALENCIA/VALÈNCIA, COMUNIDAD VALENCIANA', 'imagen': 'imagenes\\MARIA CARMEN CAMARA LOPEZ.png'}, {'NOMBRE': 'MOISES SÁNCHEZ ESCRIBANO', 'DESAPARECE': '12 de enero de 2024', 'FECHA NACIMIENTO': '06 de mayo de 2007', 'EDAD AL DESAPARECER': '16 años', 'EDAD ACTUAL': '16 años', 'LUGAR DE DESAPARICIÓN': 'LEGANES. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\MOISES SÁNCHEZ ESCRIBANO.png'}, {'NOMBRE': 'EMILY VAQUERO PALII', 'DESAPARECE': '19 de mayo de 2023', 'FECHA NACIMIENTO': '26 de agosto de 2020', 'EDAD AL DESAPARECER': '2 años', 'EDAD ACTUAL': '3 años', 'LUGAR DE DESAPARICIÓN': 'ZARAGOZA. ZARAGOZA, ARAGÓN', 'imagen': 'imagenes\\EMILY VAQUERO PALII.png'}, {'NOMBRE': 'SELENE HEREDIA BERMUDEZ', 'DESAPARECE': '15 de enero de 2024', 'FECHA NACIMIENTO': '21 de octubre de 2006', 'EDAD AL DESAPARECER': '17 años', 'EDAD ACTUAL': '17 años', 'LUGAR DE DESAPARICIÓN': 'TOLEDO. TOLEDO, CASTILLA LA MANCHA', 'imagen': 'imagenes\\SELENE HEREDIA BERMUDEZ.png'}]


In [ ]:
import pandas as pd


# Convertir la lista de diccionarios a un DataFrame
df = pd.DataFrame(desaparecidos)

# Guardar en un archivo CSV
df.to_csv('desaparecidos.csv', index=False)

# Guardar en un archivo JSON
df.to_json('desaparecidos.json', orient='records', lines=True)
